In [6]:
from google.colab import drive
drive.mount("/content/drive")

path_final = "/content/drive/MyDrive/LTSSUD"
import os

if (os.path.isdir(path_final) == True):
  %cd "/content/drive/MyDrive/LTSSUD"

!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/LTSSUD
Proposal.gdoc  train.csv   Untitled0.ipynb  voice.csv
test.csv       tree.ipynb  Untitled1.ipynb


In [7]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute  import SimpleImputer

In [30]:
# Define the loss function (e.g., mean squared error)
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)


def residuals(y_true, y_pred):
    return (y_true - y_pred)


def build_weak_learner(X, y):
    # Create the weak learner (decision tree) and fit it to the gradients and Hessians
    model = DecisionTreeRegressor(max_depth=3)  # Adjust the hyperparameters as needed
    model.fit(X, y)  # Divide gradients by Hessians to account for second-order effects
    return model

# Define the XGBoost model
class XGBoostModel:
    def __init__(self, n_estimators, learning_rate):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.initial_prediction = 0
        self.models = []

    def fit(self, X, y):
        # Convert data to DMatrix format
        #data = pd.concat([X, y], axis=1)
        data = X
        #dmatrix = DMatrix(data.values)

        # Initialize the predictions with the mean value of y
        self.initial_prediction = np.mean(y)
        predictions = np.full(len(y), self.initial_prediction)
        #print(self.initial_prediction)
        # Build the models in a loop
        for _ in range(self.n_estimators):
            # Compute the gradients and Hessians
            residual = residuals(y, predictions)

            # Fit a weak learner (e.g., a decision tree) to the gradients and Hessians
            model = build_weak_learner(data, residual)  # Implement your own weak learner

            # Update the predictions using the learning rate and the predictions of the weak learner
            predictions += self.learning_rate * model.predict(data)
            #print(predictions)

            # Add the model to the ensemble
            self.models.append(model)

    def predict(self, X):
        # Convert data to DMatrix format
        #data = DMatrix(X.values)

        # Make predictions by aggregating the predictions of all models in the ensemble
        predictions = np.full(len(X),self.initial_prediction)
        for model in self.models:
            predictions += self.learning_rate * model.predict(X)
        return predictions

In [34]:
# Example usage
dat = pd.read_csv('train.csv')
y = dat.SalePrice
X = dat.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.25)
train_X.fillna(0, inplace = True)
test_X.fillna(0, inplace = True)
my_imputer = SimpleImputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)

In [35]:
# Create and train the XGBoost model
xgb_model = XGBoostModel(n_estimators=1000, learning_rate=0.1)
xgb_model.fit(train_X, train_y)

In [36]:
# Make predictions
y_pred = xgb_model.predict(test_X)

In [37]:
from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(y_pred, test_y)))

Mean Absolute Error : 15877.062344607531


In [31]:
data = pd.DataFrame({'feature1': [1.6, 1.6, 1.5, 1.8, 1.5, 1.4],
                     'feature2': [3, 2, 3, 1, 2, 3],
                     'feature3': [1, 0, 0, 1, 1, 0],
                     'target': [88, 76, 56, 73, 77, 57]})

# Separate features and target
X = data[['feature1', 'feature2']].values
y = data['target'].values

# Create and fit an XGBoost model
xgb_model = XGBoostModel(n_estimators=100, learning_rate=0.1)
xgb_model.fit(X, y)

# Make predictions
predictions = xgb_model.predict(X)

In [32]:
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, y)))

Mean Absolute Error : 0.00025971145578769966


In [33]:
predictions

array([87.99955288, 75.99987162, 56.00040285, 72.9999513 , 76.99984506,
       57.00037629])